# **Notebook Objective**

**Description:**  
This notebook serves as the first step in the data analysis pipeline.  
Its primary purpose is to load the raw dataset (`clean_merged_data.csv`),  
perform a thorough data quality audit, engineer new features essential for  
the analysis (such as lags, rolling averages, and interaction terms), and  
then export the cleaned and enriched data to a new file named  
`analysis_ready_data.csv`. This new file will be the single source of truth  
for all subsequent analysis notebooks.

In [6]:
# Import necessary libraries
import pandas as pd
from pathlib import Path

# Define file paths
# This assumes the notebook is in the '4_data_analysis' folder
# and the data is in the '0_datasets' folder.
input_file = Path("../1_datasets/final_datasets/clean_merged_data.csv")
output_file = Path("../0_datasets/analysis_ready_data.csv")

# Ensure the output directory exists
output_file.parent.mkdir(parents=True, exist_ok=True)


### **Data Loading and Initial Inspection**

**Description:**  
This cell loads the dataset from the specified input file and performs a  
high-level initial inspection. We check the dataset's dimensions (number of  
rows and columns), list the column names, and display the first few rows to  
get a feel for the data's structure and content.

In [7]:
# Load the dataset
df = pd.read_csv(input_file)

# Display basic information
print(f"Dataset loaded from: {input_file}")
print("--- Initial Inspection ---")
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# Display the first 5 rows
print("\nFirst 5 rows of the dataset:")
display(df.head())


Dataset loaded from: ..\1_datasets\final_datasets\clean_merged_data.csv
--- Initial Inspection ---
Dataset shape: (1950, 8)
Columns: ['Country', 'Year', 'SDI', 'PM2.5', 'All-cause DALYs', 'Cardiovascular DALYs', 'Stroke DALYs', 'Respiratory DALYs']

First 5 rows of the dataset:


,Country,Year,SDI,PM2.5,All-cause DALYs,Cardiovascular DALYs,Stroke DALYs,Respiratory DALYs
0,Botswana,2010,0.585824,14.14,2628.442345,907.907901,491.239471,209.815091
1,Botswana,2011,0.591985,13.69,2631.550406,914.376036,491.661785,208.840770
2,Botswana,2012,0.597740,13.00,2565.804889,882.190695,471.917945,202.342766
3,Botswana,2013,0.604065,12.81,2531.555083,867.937928,462.699749,198.976657
4,Botswana,2014,0.610086,12.78,2478.617501,844.059254,448.100915,194.019144


### **Data Quality Audit**

**Description:**  
This cell conducts a detailed data quality assessment to identify potential  
issues. We check the data types of each column, count the number of missing  
values, check for and count any duplicate rows, and verify that the 'Year'  
column falls within our expected range (2010-2019).

In [8]:
# Perform a detailed data quality audit
print("--- Data Quality Audit ---")

# Check data types and non-null counts
print("\nData Types and Non-Null Counts:")
df.info()

# Check for missing values
print("\nMissing Values per Column:")
print(df.isnull().sum())

# Check for duplicate rows
num_duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows found: {num_duplicates}")

# Check for years outside the expected range
invalid_years_count = df[
    ~df['Year'].between(2010, 2019)
].shape[0]
print(
    f"Number of rows with Year outside 2010–2019: {invalid_years_count}"
)

--- Data Quality Audit ---

Data Types and Non-Null Counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950 entries, 0 to 1949
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               1950 non-null   object 
 1   Year                  1950 non-null   int64  
 2   SDI                   1950 non-null   float64
 3   PM2.5                 1950 non-null   float64
 4   All-cause DALYs       1950 non-null   float64
 5   Cardiovascular DALYs  1950 non-null   float64
 6   Stroke DALYs          1950 non-null   float64
 7   Respiratory DALYs     1950 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 122.0+ KB

Missing Values per Column:
Country                 0
Year                    0
SDI                     0
PM2.5                   0
All-cause DALYs         0
Cardiovascular DALYs    0
Stroke DALYs            0
Respiratory DALYs       0
dtype: int64

Number of 

### **Feature Engineering**

**Description:**  
In this crucial step, we engineer new features to enhance the dataset  
for modeling. This includes:

1. **Sorting** the data by Country and Year to ensure correct time-series calculations.
2. Creating **lag features** (`PM25_lag1`, `PM25_lag2`) to capture the effect  
   of past pollution.
3. Creating **rolling average features** (`PM25_3yr_avg`, `PM25_5yr_avg`)  
   to model cumulative exposure.
4. Creating an **interaction term** between PM₂.₅ and SDI to test if
   development level modifies the pollution-health relationship.
5. Creating a categorical variable for **SDI** to enable stratified analysis  
   in later notebooks.

In [9]:
# --- Feature Engineering ---
print("Starting feature engineering...")

# Ensure data is sorted for time-series operations
df = df.sort_values(['Country', 'Year']).reset_index(drop=True)

# 1. Create 1-year and 2-year lag features for PM2.5
df['PM25_lag1'] = df.groupby('Country')['PM2.5'].shift(1)
df['PM25_lag2'] = df.groupby('Country')['PM2.5'].shift(2)

# 2. Compute 3-year and 5-year rolling averages for PM2.5
df['PM25_3yr_avg'] = (
    df.groupby('Country')['PM2.5']
      .rolling(window=3, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)
df['PM25_5yr_avg'] = (
    df.groupby('Country')['PM2.5']
      .rolling(window=5, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)

# 3. Create interaction term: PM2.5 * SDI
df['PM25_SDI_interaction'] = df['PM2.5'] * df['SDI']

# 4. Create SDI categories for stratification
sdi_bins = [0.0, 0.45, 0.61, 0.75, 1.0]
sdi_labels = ['Low', 'Medium', 'High', 'Very High']
df['SDI_category'] = pd.cut(
    df['SDI'],
    bins=sdi_bins,
    labels=sdi_labels,
    include_lowest=True
)

print("Feature engineering complete.")

# Verify the new features by displaying a sample
print("\nSample of DataFrame with new features:")
new_features = [
    'PM25_lag1', 'PM25_lag2', 'PM25_3yr_avg', 'PM25_5yr_avg',
    'PM25_SDI_interaction', 'SDI_category'
]
display(df[['Country', 'Year', 'PM2.5', 'SDI'] + new_features].head(7))


Starting feature engineering...
Feature engineering complete.

Sample of DataFrame with new features:


,Country,Year,PM2.5,SDI,PM25_lag1,PM25_lag2,PM25_3yr_avg,PM25_5yr_avg,PM25_SDI_interaction,SDI_category
0,Afghanistan,2010,68.97,0.247760,NaN,NaN,68.970000,68.970000,17.088004,Low
1,Afghanistan,2011,66.94,0.257042,68.97,NaN,67.955000,67.955000,17.206370,Low
2,Afghanistan,2012,68.26,0.266484,66.94,68.97,68.056667,68.056667,18.190226,Low
3,Afghanistan,2013,72.18,0.275637,68.26,66.94,69.126667,69.087500,19.895447,Low
4,Afghanistan,2014,68.06,0.284030,72.18,68.26,69.500000,68.882000,19.331106,Low
5,Afghanistan,2015,67.20,0.291850,68.06,72.18,69.146667,68.528000,19.612287,Low
6,Afghanistan,2016,64.00,0.299631,67.20,68.06,66.420000,67.940000,19.176365,Low


### **Export Processed Data**

**Description:**  
This final cell saves the fully cleaned and feature-enriched DataFrame  
to a new CSV file. This `analysis_ready_data.csv` file will serve as  
the input for all subsequent analytical notebooks, ensuring consistency  
and reproducibility.

In [10]:
# Export the processed DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print("--- Export Complete ---")
print(
    f"Processed data with {df.shape[1]} columns saved to: {output_file}"
)
print(f"Final shape of the analysis-ready data: {df.shape}")


--- Export Complete ---
Processed data with 14 columns saved to: ..\0_datasets\analysis_ready_data.csv
Final shape of the analysis-ready data: (1950, 14)
